In [ ]:
import os
import re
import pandas as pd
import GladUtil as gd

ym  = '202212'
dir0  = f'e:/글로벌백업(180329)/04.수수료작업/{ym}수수료/'
fileN = f'{ym}_수입원장.xlsx'
cols = ['지점','직할지점','사원번호','사원명','수수료계','계약일자','게약종류']
print('reading ',fileN)

df = pd.read_excel(dir0+fileN, sheet_name='전체', usecols=cols, dtype={'계약일자': str})

# -- 지점변환
f = r'e:\글로벌백업(180329)\77.EXCEL\My VBA\KJ정산작업Util_4.0.xlsm'
s = '정산작업'
df1 = pd.read_excel(f, sheet_name=s, skiprows=5, usecols='i,j,l,n,p,r')
d1_brh  = df1.iloc[:,[0,1]].dropna()
d1_team = df1.iloc[:,[2,3]].dropna()
d1_fc   = df1.iloc[:,[4,5]].dropna()
d1 = dict(zip(d1_brh.iloc[:,0]  ,d1_brh.iloc[:,1]))
d2 = dict(zip(d1_team.iloc[:,0] ,d1_team.iloc[:,1]))
d3 = dict(zip(d1_fc.iloc[:,0]   ,d1_fc.iloc[:,1]))

df['지점'].fillna('글래드',inplace=True)
df['변환지점'] = df.apply(lambda x: d1.get(x['지점'],x['지점']),axis=1)
df['변환지점'] = df.apply(lambda x: d2.get(x['직할지점'],x['변환지점']),axis=1)
df['변환지점'] = df.apply(lambda x: d3.get(x['사원번호'],x['변환지점']),axis=1)
# -- 요약
df_sum = df.groupby(['변환지점','계약종류'])['수수료계'].sum().reset_index()
df_sum.rename(columns = {'지점':'원지점','변환지점':'지점'}, inplace=True)
df_sum.loc[:, "지사"] = df_sum.지점.map(lambda x: '지사'  if x in gd.GROUP2 
                                    else ('직영' if x in gd.GROUP1 else '오류'))


In [ ]:

file_list = os.listdir(dir0)
# print(file_list)
file_list2= [ s for s in file_list if "마감실적리스트" in s]
dfAll = pd.DataFrame()

for fname in file_list2: 
    bojong = ''.join(re.findall('\(([^)]+)',  fname))
    print(bojong,' ', fname)
    df1 = pd.read_excel(f'{dir0}{fname})
    df1 = df1[df1['본부'] == 'GLAD']
    dfT = df1.groupby(['지점'])[['수수료H','수수료S']].sum().reset_index()
    dfT['계약종류'] = bojong
    dfAll = pd.concat([dfAll,dfT])

dfAll.loc[:, "지사"] = dfAll.지점.map(lambda x: '지사'  if x in gd.GROUP2 
                                    else ('직영' if x in gd.GROUP1 else '오류'))
dfAll.to_clipboard()

##### 시상금

In [176]:
dir1  = f'e:/글로벌백업(180329)/04.수수료작업/{ym}시상/'
fileN = f'{ym} KJ시상정산.xlsx'
cols = ['지점','장기법인','생보법인','일반','자동차','장기수입','장기지급','생보수입','생보지급']
print('reading ',fileN)

df3 = pd.read_excel(dir1+fileN, sheet_name='정산', usecols='b,e,h,i,j,k:n',skiprows=22)
df3.columns = cols
df3 = df3.iloc[:df3[df3['지점']=='계'].index[0]].dropna()
df3 = df3[df3['지점'].str.contains('글래드')]
df3['손보시상'] = df3['장기법인'] + df3['일반'] + df3['자동차'] + \
                  df3['장기수입'] - df3['장기지급'] 
df3['생보시상'] = df3['생보법인'] + df3['생보수입'] - df3['생보지급'] 
df3[['지점','손보시상','생보시상']].to_clipboard()

reading  202212 KJ시상정산.xlsx


In [175]:
df3['손보시상'].sum() + df3['생보시상'].sum() 

26497908.847555242

In [177]:
dfAll.to_clipboard()